In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import sys
import matplotlib.pyplot as plt
from tsne_torch import TorchTSNE
import numpy as np
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.9/site-packages/')
from pytorch_metric_learning import losses

sys.path.append('/home/gridsan/ckoevesdi/PooledStatisticsMetamers/poolstatmetamer/')
import utils.statnetencoder as sne
import importlib
import imp
import torch.nn.functional as F

#!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py

#sys.path.append(r'C:\Users\chris\Documents\MIT\Statistics_analysis_code\PyTorchSteerablePyramid')
sys.path.append('/home/gridsan/ckoevesdi/PyTorchSteerablePyramid/')
import steerable
import steerable.utils as utils
from steerable.SCFpyr_PyTorch import SCFpyr_PyTorch
sys.path.append('/home/gridsan/ckoevesdi/Testobreal/')

torch.manual_seed(16)

#use GPU 2
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
import utils.brucenet as bn
brucy = bn.BruceNet(pooling_region_size=1e20, pyramid_params=False, dummy_img = torch.zeros(20,
                                     1,
                                     630,
                                     840)).to(device)

Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
# Path to the image
from PIL import Image
image_path = "/home/gridsan/ckoevesdi/Testobreal/44.jpg"

# Read the image
x = Image.open(image_path)
x_gray = x.convert("L")
# Convert the PIL Image to a NumPy array
image_array = np.array(x_gray)
to_tensor = transforms.ToTensor()
x_tensor = to_tensor(x_gray)
xx = torch.stack([x_tensor, x_tensor]).to(device)

In [4]:
# Path to the image
from PIL import Image
image_path = "/home/gridsan/ckoevesdi/Testobreal/43.jpg"

# Read the image
y = Image.open(image_path)
y = y.convert("L")

# Convert the PIL Image to a NumPy array
image_array = np.array(y)
to_tensor = transforms.ToTensor()
y_tensor = to_tensor(y)
yy = torch.stack([y_tensor, y_tensor]).to(device)
print(yy.shape)

torch.Size([2, 1, 630, 840])


In [5]:
# Path to the image
from PIL import Image
image_path = "/home/gridsan/ckoevesdi/Testobreal/orig.jpg"

# Read the image
y = Image.open(image_path)
y = y.convert("L")

# Convert the PIL Image to a NumPy array
image_array = np.array(y)
to_tensor = transforms.ToTensor()
y_tensor = to_tensor(y)
zz = torch.stack([y_tensor, y_tensor]).to(device)
print(zz.shape)

torch.Size([2, 1, 630, 840])


In [6]:
statistics3 = brucy(zz)  # This should output a tensor of shape [2, 150]
statistics3 = statistics3[0,:]
print(statistics3)

tensor([ 1.7058e+00,  8.0517e-01,  8.1627e-02,  1.1353e+00,  1.1008e+00,
         1.0808e+00,  1.5949e+00,  1.5414e+00,  1.5111e+00,  2.2290e+00,
         2.1419e+00,  2.0943e+00,  3.1081e+00,  2.9692e+00,  2.8972e+00,
         4.3199e+00,  4.1002e+00,  3.9893e+00,  3.6685e-01,  3.4274e-01,
         5.1597e-01,  9.8224e-02,  2.3651e-01,  1.3014e-01,  1.4534e-01,
         3.8183e-02,  2.3796e-01,  1.7185e-01,  2.7811e-01,  5.3021e-02,
         3.9533e-01,  3.4713e-01,  5.9272e-01,  3.9181e-02,  1.6927e-01,
         1.1434e-01,  2.8802e-01,  1.1193e-01,  1.7591e-01,  1.7174e-01,
         7.1330e-01,  7.7737e-01,  9.3105e-01,  3.1284e-01,  3.5216e-01,
         1.8018e-01,  2.2466e-01,  1.2615e-01,  4.9931e-01,  4.3140e-01,
         5.7732e-01,  1.6890e-01,  8.7638e-01,  1.1124e+00,  1.1984e+00,
         4.5787e-01,  2.8173e-01,  2.8274e-01,  7.8357e-01,  2.0124e-01,
         3.0736e-01,  5.0111e-01,  1.1278e+00,  1.1807e+00,  1.3282e+00,
         4.5527e-01,  6.0029e-01,  2.8542e-01,  3.6

In [7]:
statistics = brucy(yy)  # This should output a tensor of shape [2, 150]
statistics = statistics[0,:]

In [8]:
statistics2 = brucy(xx)  # This should output a tensor of shape [2, 150]
statistics2 = statistics2[0,:]

In [ ]:
print(statistics2)

In [ ]:
print(statistics)

In [15]:
import torch
import torch.nn.functional as F


# Slice the vectors to get the first 50 and last 50 elements
yy_first50 = statistics[20:50]
zz_first50 = statistics3[20:50]
yy_last50 = statistics[100:130]
zz_last50 = statistics3[100:130]

# Compute the MSE between the first 50 elements of each vector
mse_first50 = F.mse_loss(yy_first50, zz_first50)

# Compute the MSE between the last 50 elements of each vector
mse_last50 = F.mse_loss(yy_last50, zz_last50)

# Print the results
print(f"MSE for first 50 elements: {mse_first50.item()}")
print(f"MSE for last 50 elements: {mse_last50.item()}")


MSE for first 50 elements: 0.0008842774550430477
MSE for last 50 elements: 0.006435114424675703


In [12]:
import torch
import torch.nn.functional as F


# Slice the vectors to get the first 50 and last 50 elements
yy_first50 = statistics[:50]
zz_first50 = statistics3[:50]
yy_last50 = statistics[50:100]
zz_last50 = statistics3[50:100]

# Compute the MSE between the first 50 elements of each vector
mse_first50 = F.mse_loss(statistics, statistics3)

# Compute the MSE between the last 50 elements of each vector
mse_last50 = F.mse_loss(statistics2, statistics3)

# Print the results
print(f"MSE for first 50 elements: {mse_first50.item()}")
print(f"MSE for last 50 elements: {mse_last50.item()}")


MSE for first 50 elements: 0.0041005113162100315
MSE for last 50 elements: 0.0033451493363827467


In [ ]:
print(yy)